In [3]:
import pandas as pd
import datetime
import os
from scipy.sparse import lil_matrix
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF

In [50]:
dfm = pd.read_csv('data/dataset15.csv', sep='\t')
dfm['date'] = dfm.Time.map(lambda x:datetime.date.fromtimestamp(x))
print(len(dfm))
dfm.head(3)

45881450


,UserId,ItemId,SessionId,Time,date
0,1,181459,0,1.444342e+09,2015-10-09
1,1,779078,0,1.444342e+09,2015-10-09
2,1,779078,0,1.444342e+09,2015-10-09


In [5]:
tar_dfm = dfm[['UserId', 'ItemId']]
tar_dfm.head(2)

,UserId,ItemId
0,1,181459
1,1,779078


In [6]:
User = sorted(tar_dfm.UserId.unique())
print(len(User), User[:3], User[-3:])

Item = sorted(tar_dfm.ItemId.unique())
print(len(Item), Item[:3], Item[-3:])

422282 [1, 2, 3] [424168, 424169, 424170]
624221 [2, 3, 4] [1113162, 1113163, 1113166]


In [7]:
User2No = dict([(User[i], i) for i in range(len(User))])
Item2No = dict([(Item[i], i) for i in range(len(Item))])

In [8]:
m = lil_matrix((len(User), len(Item)))

In [9]:
for row in tar_dfm.itertuples():
    m[User2No[row.UserId], Item2No[row.ItemId]] = 1

In [15]:
M = m.tocsr()

In [16]:
N_COMPONENTS = 5
# NMF_INIT = 'nndsvd'
NMF_INIT = 'random'
NMF_SOLVER = 'cd'  # Coordinate Descent solver.
NMF_TOL = 0.0001
NMF_MAX_ITER = 200
NMF_RANDOM_STATE = 0
NMF_ALPHA = 0.0
NMF_L1_RATIO = 0.0
NMF_SHUFFLE = True

In [17]:
model = NMF(n_components=N_COMPONENTS,
            init=NMF_INIT,
            solver=NMF_SOLVER,
            tol=NMF_TOL,
            max_iter=NMF_MAX_ITER,
            random_state=NMF_RANDOM_STATE,
            alpha=NMF_ALPHA,
            l1_ratio=NMF_L1_RATIO,
            shuffle=False)

In [18]:
W = model.fit_transform(M)

In [19]:
H = model.components_

In [20]:
W.shape

(422282, 5)

In [45]:
U2Cls = {}
for i, row in enumerate(W):
    # if i>1000:break
    #print(row, row.argmax(), row[row.argmax()])
    U2Cls[User[i]] = row.argmax()

In [46]:
Rows = []
for user, c in U2Cls.items():
    Rows.append((user,c))
u2cls_dfm = pd.DataFrame(Rows)    
u2cls_dfm.columns = ['user_id', 'class']

In [47]:
class2count = pd.DataFrame(u2cls_dfm.groupby('class').count()).reset_index()
class2count.columns = ['class', 'count']
class2count

,class,count
0,0,117802
1,1,100071
2,2,150278
3,3,15955
4,4,38176


In [49]:
Cls2Users = {}
for user, cls in U2Cls.items():
    Cls2Users.setdefault(cls, [])
    Cls2Users[cls].append(user)
len(Cls2Users)    

5

In [51]:
Cls2Dfm = {}
for cls, Users in Cls2Users.items():
    Cls2Dfm[cls] = dfm[dfm.UserId.isin(Cls2Users[cls])]

In [52]:
for cls, c_dfm in Cls2Dfm.items():
    print(cls, len(c_dfm))

2 11854401
0 18639032
4 4105759
1 9254761
3 2027497


In [61]:
start_date = datetime.date(2015,5,20)
end_date = datetime.date(2015,11,12)

target_start_date = start_date + datetime.timedelta(28)

AllRows = []
for cls, c_dfm in Cls2Dfm.items():
    RF2N = {}
    RF2H = {}
    for i in range((end_date - target_start_date).days + 1):
        target_date = target_start_date + datetime.timedelta(i)
        train_start_date = target_date - datetime.timedelta(28) 
        train_end_date = target_date - datetime.timedelta(1) 
        #if i>1:break
        if i>146:break
        print(i, train_start_date.strftime("%F"), '~', train_end_date.strftime("%F"), ':', target_date.strftime("%F"), flush=True)  
        x_dfm = c_dfm[(train_start_date <= c_dfm.date) & (c_dfm.date <= train_end_date)].copy()
        y_dfm = c_dfm[c_dfm.date == target_date].copy()
        x_record_num = len(x_dfm)
        y_record_num = len(y_dfm)
        print(x_record_num, y_record_num)
        x_dfm['target_date'] = target_date
        x_dfm['day_rcen'] = (x_dfm.target_date - x_dfm.date).map(lambda x:x.days)

        U2Is = {}
        for row in y_dfm.itertuples():
            U2Is.setdefault(row.UserId, set())
            U2Is[row.UserId].add(row.ItemId)

        U2I2Info = {}
        for row in x_dfm.itertuples():
            U2I2Info.setdefault(row.UserId, {})
            U2I2Info[row.UserId].setdefault(row.ItemId, [])
            U2I2Info[row.UserId][row.ItemId].append(row.day_rcen)

        for UserId, I2Info in U2I2Info.items():
            HitSet = set(U2Is[UserId]) if UserId in U2Is else set()
            for ItemId, Info in I2Info.items():
                freq = len(Info)
                rcen = min(Info)
                RF2N.setdefault((rcen,freq), 0)
                RF2H.setdefault((rcen,freq), 0)
                RF2N[rcen,freq] += 1
                if ItemId in HitSet:
                    RF2H[rcen,freq] += 1
        N = 0
        for rf, n in RF2N.items():
            N += n
        print('data:', N)
    RF2P = {}
    for rf, n in RF2N.items():
        RF2P[rf] = RF2H[rf] / n                        

    RFRows = []
    for rf, n in RF2N.items():
        h = RF2H[rf]
        p = RF2P[rf]
        RFRows.append((rf[0], rf[1＝], h, n , p))
        AllRows.append((cls, rf[0], rf[1], h, n , p))
        
    rf_dfm = pd.DataFrame(RFRows)
    rf_dfm.columns = ['rcen', 'freq', 'hit', 'N', 'prob']
    rf_dfm = rf_dfm.sort_values(['rcen', 'freq'])
    rf_dfm = rf_dfm[rf_dfm.freq <= 40]
    fname = 'output_cls/rf_cls%02d.csv' % cls
    rf_dfm.to_csv(fname, index=False)
all_dfm = pd.DataFrame(AllRows)
all_dfm.columns = ['cls', 'rcen', 'freq', 'hit', 'N', 'prob']
all_dfm = all_dfm.sort_values(['cls', 'rcen', 'freq'])
all_dfm = all_dfm[all_dfm.freq <= 40]
fname = 'output_cls/cls_rf.csv'
all_dfm.to_csv(fname, index=False)

0 2015-05-20 ~ 2015-06-16 : 2015-06-17
1042020 65830
data: 680022
1 2015-05-21 ~ 2015-06-17 : 2015-06-18
1067028 48773
data: 1373685
2 2015-05-22 ~ 2015-06-18 : 2015-06-19
1073141 39429
data: 2069492
3 2015-05-23 ~ 2015-06-19 : 2015-06-20
1063617 37872
data: 2760009
4 2015-05-24 ~ 2015-06-20 : 2015-06-21
1058148 35694
data: 3447177
5 2015-05-25 ~ 2015-06-21 : 2015-06-22
1052059 34690
data: 4130432
6 2015-05-26 ~ 2015-06-22 : 2015-06-23
1047828 31575
data: 4811513
7 2015-05-27 ~ 2015-06-23 : 2015-06-24
1041767 38144
data: 5489054
8 2015-05-28 ~ 2015-06-24 : 2015-06-25
1040648 42840
data: 6166478
9 2015-05-29 ~ 2015-06-25 : 2015-06-26
1042244 76926
data: 6846104
10 2015-05-30 ~ 2015-06-26 : 2015-06-27
1079976 54319
data: 7547901
11 2015-05-31 ~ 2015-06-27 : 2015-06-28
1097700 47856
data: 8260759
12 2015-06-01 ~ 2015-06-28 : 2015-06-29
1111524 30477
data: 8982027
13 2015-06-02 ~ 2015-06-29 : 2015-06-30
1109041 27042
data: 9701582
14 2015-06-03 ~ 2015-06-30 : 2015-07-01
1104300 32866
data: